In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import joblib
import random
import numpy as np


In [75]:
data = pd.read_csv('availability.csv')
data.rename(columns={'last_update': 'time_of_day'}, inplace=True)
data

,station_id,time_of_day,available_bike_stands,bike_stands,available_bikes,status
0,1,2024-02-21 22:18:52,9,31,22,OPEN
1,1,2024-02-21 22:37:12,8,31,23,OPEN
2,1,2024-02-21 22:47:17,8,31,23,OPEN
3,1,2024-02-21 22:57:38,7,31,24,OPEN
4,1,2024-02-21 23:00:01,8,31,23,OPEN
...,...,...,...,...,...,...
565009,117,2024-03-20 22:19:17,29,40,11,OPEN
565010,117,2024-03-20 22:23:42,29,40,11,OPEN
565011,117,2024-03-20 22:28:23,29,40,11,OPEN
565012,117,2024-03-20 22:38:28,29,40,11,OPEN


In [76]:

data2 = pd.read_csv('weather.csv')
data2

,time_of_day,station_id,main,description,temperature,wind_speed
0,2024-02-25 15:03:04,1,Clouds,broken clouds,8,9.77
1,2024-02-25 15:03:04,9,Clouds,broken clouds,8,9.77
2,2024-02-25 15:03:04,14,Clouds,broken clouds,8,9.77
3,2024-02-25 15:03:04,27,Clouds,broken clouds,8,9.77
4,2024-02-25 15:03:04,36,Clouds,broken clouds,8,9.77
...,...,...,...,...,...,...
198240,2024-03-20 22:40:19,106,Clear,clear sky,5,4.12
198241,2024-03-20 22:40:19,107,Clear,clear sky,6,4.12
198242,2024-03-20 22:40:20,108,Clear,clear sky,5,4.12
198243,2024-03-20 22:40:20,112,Clear,clear sky,6,4.12


In [77]:
import pandas as pd

# Load your two dataframes
df1 = data
df2 = data2
# Convert 'time_of_day' column to datetime format
df1['time_of_day'] = pd.to_datetime(df1['time_of_day'])
df2['time_of_day'] = pd.to_datetime(df2['time_of_day'])
# Sort dataframes by 'time_of_day'
df1.sort_values('time_of_day', inplace=True)
df2.sort_values('time_of_day', inplace=True)

# Merge dataframes using merge_asof
merged_df = pd.merge_asof(df1, df2, by='station_id', on='time_of_day', direction="nearest")




In [78]:
merged_df.head(1500)

,station_id,time_of_day,available_bike_stands,bike_stands,available_bikes,status,main,description,temperature,wind_speed
0,72,2024-02-21 22:09:43,31,31,0,OPEN,Clouds,broken clouds,8,9.77
1,73,2024-02-21 22:09:46,20,30,10,OPEN,Clouds,broken clouds,8,9.77
2,26,2024-02-21 22:16:21,20,20,0,OPEN,Clouds,broken clouds,8,9.77
3,54,2024-02-21 22:18:11,33,33,0,OPEN,Clouds,broken clouds,8,9.77
4,16,2024-02-21 22:18:20,1,20,19,OPEN,Clouds,broken clouds,8,9.77
...,...,...,...,...,...,...,...,...,...,...
1495,67,2024-02-22 00:16:36,30,40,10,OPEN,Clouds,broken clouds,8,9.77
1496,111,2024-02-22 00:16:38,33,40,7,OPEN,Clouds,broken clouds,7,9.77
1497,53,2024-02-22 00:16:40,40,40,0,OPEN,Clouds,broken clouds,8,9.77
1498,62,2024-02-22 00:16:40,32,40,8,OPEN,Clouds,broken clouds,8,9.77


In [82]:
data = merged_df

data['month'] = pd.to_datetime(data['time_of_day']).dt.month
data['day'] = pd.to_datetime(data['time_of_day']).dt.day
data['hour'] = pd.to_datetime(data['time_of_day']).dt.hour
data['minute'] = pd.to_datetime(data['time_of_day']).dt.minute
data = pd.get_dummies(data, columns=['main', "description"], prefix='category')

data = data.drop(columns=["status", "time_of_day"])
X = data.drop(columns = ['available_bikes',"available_bike_stands"])  # Features
Y = data['available_bikes']  # Target variable

X


,station_id,bike_stands,temperature,wind_speed,month,day,hour,minute,category_Clear,category_Clouds,...,category_light intensity drizzle,category_light intensity drizzle rain,category_light intensity shower rain,category_light rain,category_mist,category_moderate rain,category_overcast clouds,category_scattered clouds,category_sleet,category_snow
0,72,31,8,9.77,2,21,22,9,False,True,...,False,False,False,False,False,False,False,False,False,False
1,73,30,8,9.77,2,21,22,9,False,True,...,False,False,False,False,False,False,False,False,False,False
2,26,20,8,9.77,2,21,22,16,False,True,...,False,False,False,False,False,False,False,False,False,False
3,54,33,8,9.77,2,21,22,18,False,True,...,False,False,False,False,False,False,False,False,False,False
4,16,20,8,9.77,2,21,22,18,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565009,28,30,6,4.12,3,20,22,49,True,False,...,False,False,False,False,False,False,False,False,False,False
565010,108,35,5,4.12,3,20,22,49,True,False,...,False,False,False,False,False,False,False,False,False,False
565011,56,40,6,4.12,3,20,22,49,True,False,...,False,False,False,False,False,False,False,False,False,False
565012,90,40,6,4.12,3,20,22,49,True,False,...,False,False,False,False,False,False,False,False,False,False


In [83]:
Y

0          0
1         10
2          0
3          0
4         19
          ..
565009     4
565010     8
565011    13
565012     3
565013     0
Name: available_bikes, Length: 565014, dtype: int64

In [84]:
# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=52)


In [85]:
# Initialize Naive Bayes classifier
model = GaussianNB()

# Train the model
model.fit(X_train, y_train)


GaussianNB()

In [86]:
y_pred = model.predict(X_test)

In [87]:
from sklearn import linear_model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 383.27756785218094
R-squared: -2.8257815384781226


In [88]:


reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [89]:
reg.coef_

array([-1.16325424e-02,  2.89227848e-01, -5.67022599e-02, -7.07149802e-03,
       -3.99414782e-01, -1.45225005e-02, -5.57338317e-03,  3.24729003e-04,
        2.62499792e-01,  2.75076480e-01,  2.56817924e-01, -1.80920577e+00,
        3.32825705e-01,  4.56590733e-01,  2.25395140e-01,  4.84689931e-02,
        2.62499792e-01, -7.42106634e-02, -1.80920577e+00,  4.19786611e-01,
       -1.14146609e-01,  3.70964533e-01,  1.42998959e-01,  2.13277648e-02,
        3.32825705e-01, -1.27522601e-01,  2.85262872e-01,  1.55552776e-02,
        1.24567391e-01,  1.00827750e-01])

In [90]:
# Predictions on the testing set
y_pred = model.predict(X_test)


In [91]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 383.27756785218094
R-squared: -2.8257815384781226


In [92]:
# Predictions on the testing set
y_pred = model.predict(X_test)


In [93]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.03222923285222516
